# Descrição Dataframe

## Amostra
Segue uma pequena amostra do dataframe já tratado

In [1]:
import pandas as pd

df = pd.read_csv("dados/amostra.csv")

df["dt_despacho_pedido"] = pd.to_datetime(df["dt_despacho_pedido"])

df["dt_entrega_pedido"] = pd.to_datetime(df["dt_entrega_pedido"])

df["dt_previsao_entrega_cliente"] = pd.to_datetime(df["dt_previsao_entrega_cliente"])

df["dt_criacao"] = pd.to_datetime(df["dt_criacao"])

df["dt_pagamento_pedido"] = pd.to_datetime(df["dt_pagamento_pedido"])

df["flg_existem_ocorrencias"] = df["flg_existem_ocorrencias"].str.lower() == "sim"

df["entregue_no_prazo"] = df["tp_performance_entrega"] == "Entregue no Prazo"

df["qtde_dias_prazo_entrega"] = (
    df["dt_previsao_entrega_cliente"] - df["dt_pagamento_pedido"]
).dt.days

df["qtde_dias_efet_entrega"] = (
    df["dt_entrega_pedido"] - df["dt_pagamento_pedido"]
).dt.days

df["qtde_dias_demora_despacho"] = (
    df["dt_despacho_pedido"] - df["dt_pagamento_pedido"]
).dt.days

df["pct_atraso"] = df.apply(
    lambda row: (
        row["qtde_dias_efet_entrega"] / row["qtde_dias_prazo_entrega"]
        if row["qtde_dias_prazo_entrega"] > 0
        else row["qtde_dias_efet_entrega"]
    ),
    axis=1,
)
df["pct_atraso"] = df["pct_atraso"].clip(lower=0)

df = df.query(
    "not dt_despacho_pedido.isnull() and not dt_entrega_pedido.isnull() and not dt_pagamento_pedido.isnull()"
)

df.drop(
    columns=[
        "Unnamed: 0",
        "hr_despacho_pedido",
        "hr_entrega_pedido",
        "tp_performance_entrega",
        "cod_pedido",
    ],
    inplace=True,
)

df

,cidade_destinatario,uf,grp_transportadora,dt_despacho_pedido,dt_entrega_pedido,dt_previsao_entrega_cliente,dt_criacao,dt_pagamento_pedido,flg_existem_ocorrencias,tp_praca,des_unidade_negocio,des_cd_origem,qtd_dias_tat,entregue_no_prazo,qtde_dias_prazo_entrega,qtde_dias_efet_entrega,qtde_dias_demora_despacho,pct_atraso
0,CURITIBA,PR,Transportadora 4,2023-12-04,2023-12-07,2023-12-07,2023-11-30,2023-11-30,False,Capital,Multi,PR-Campina G. Sul,5.0,True,7.0,7.0,4.0,1.000000
1,PARAUAPEBAS,PA,Transportadora 2,2023-11-21,2023-12-01,2023-12-11,2023-11-15,2023-11-17,False,Interior,Mono,PR-Campina G. Sul,10.0,True,24.0,14.0,4.0,0.583333
2,JACAREZINHO,PR,Transportadora 4,2023-10-28,2023-10-31,2023-11-06,2023-10-26,2023-10-27,False,Interior,Mono,PR-Campina G. Sul,2.0,True,10.0,4.0,1.0,0.400000
3,SÃO PAULO,SP,Transportadora 5,2023-08-23,2023-08-24,2023-08-24,2023-08-22,2023-08-22,False,Capital,Multi,PR-Campina G. Sul,2.0,True,2.0,2.0,1.0,1.000000
4,SAO LUIS,MA,Transportadora 3,2023-08-14,2023-08-23,2023-08-24,2023-08-13,2023-08-12,False,Capital,Multi,PR-Campina G. Sul,8.0,True,12.0,11.0,2.0,0.916667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503523,BELO HORIZONTE,MG,Transportadora 3,2023-09-04,2023-09-07,2023-09-06,2023-09-03,2023-09-03,True,Capital,Multi,PR-Campina G. Sul,3.0,False,3.0,4.0,1.0,1.333333
503525,SÃO PAULO,SP,Transportadora 5,2023-08-23,2023-08-26,2023-08-24,2023-08-22,2023-08-22,True,Capital,Multi,PR-Campina G. Sul,3.0,False,2.0,4.0,1.0,2.000000
503526,São Paulo,SP,Transportadora 1,2023-09-19,2023-09-25,2023-09-20,2023-09-18,2023-09-16,True,Capital,Multi,SP-Jaguaré,6.0,False,4.0,9.0,3.0,2.250000
503527,NITERÓI,RJ,Transportadora 3,2023-10-16,2023-10-19,2023-10-18,2023-10-14,2023-10-14,False,Reg. Metropolitana,Mono,SP-Registro,4.0,False,4.0,5.0,2.0,1.250000


## Premissas
Segue premissas tomadas no entendimento dos dados:
- O prazo de entrega do pedido conta a partir da data de pagamento do mesmo

## Tratamentos
Segue tratamentos efetuados em cima do dataset:

- Foi retirado do dataframe os registros cuja **data despacho** esteja vazia.
- Foi retirado do dataframe os registros cuja **data entrega** esteja vazia.
- Foi retirado do dataframe os registros cuja **data pagamento** esteja vazia.
- Criado campo `pct_atraso`: Corresponde a relação entre a data efetiva da entrega e a sua previsão. Valores menores que 1 indicam que o pedido foi entregue antes do prazo e valores maiores que 1 indicam que o pedido foi entregue depois do prazo.
- Criado campos que expressam diferenças entre datas, sendo elas:
  - `qtde_dias_prazo_entrega`: Diferença entre a data prevista de entrega e a data de pagamento do pedido.
  - `qtde_dias_efet_entrega`:  Diferença entre a data efetiva de entrega e a data de pagamento do pedido.
  - `qtde_dias_demora_despacho`: Diferença entre a data de despacho e a data de pagamento do pedido.
  
## Descrição das colunas

- `cidade_destinatario`: Cidade de destino do pedido.
- `uf`: Estado da cidade destino.
- `grp_transportadora`: (Grupo) transportado que está realizando a entrega.
- `dt_despacho_pedido`: data e hora em que o pedido foi despachado.
- `dt_entrega_pedido`: data e hora em que o pedido foi entregue.
- `dt_previsao_entrega_cliente`: data prevista para a entrega do pedido.
- `dt_criacao`: data de cria o do pedido.
- `dt_pagamento_pedido`: data do pagamento do pedido.
- `flg_existem_ocorrencias`: booleano que indica se houve ocorrencias durante a entrega do pedido (problema no andamento da entrega).
- `tp_praca`: Tipo de praça de entrega (Capital, Interior, Região Metropolitana).
- `des_unidade_negocio`: Classificação da unidade de negócio.
- `des_cd_origem`: Centro de distribuição de origem.
- `entregue_no_prazo`: booleano que indica se o pedido foi entregue no prazo.
- `qtde_dias_prazo_entrega`: quantidade de dias entre a **data de despacho** e a **data de entrega prevista**.
- `qtde_dias_efet_entrega`: quantidade de dias entre a **data de despacho** e a **data de entrega efetiva**.

# Informações gerais

In [2]:
import locale
from IPython.display import Markdown

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

# Cálculo do percentual de entregas fora do prazo
percentual = (len(df.query("entregue_no_prazo == False")) / len(df)) * 100

display(
    Markdown(
        f"Percentual de entregas fora do prazo: {locale.format_string('%.2f %%', percentual, grouping=True)}"
    )
)

# Percentual médio de atraso por centro de distribuição
display(
    Markdown(
        f"Percentual médio de atraso: {locale.format_string('%.2f %%', df.query('entregue_no_prazo == False')['pct_atraso'].mean(), grouping=True)}"
    )
)

# Lista de Centro de distribuição (origem da entrega)
display(
    Markdown(
        f"Centros de distribuições disponíveis: {', '.join(df['des_cd_origem'].unique())}"
    )
)

# Transportadoras de cada centro de distribuição
display(Markdown("Transportadoras de cada centro de distribuição:"))
for centros_dist in df["des_cd_origem"].unique():
    lista_transportadoras = df.query("des_cd_origem == @centros_dist")[
        "grp_transportadora"
    ].unique()
    lista_transportadoras.sort()

    display(Markdown(f"- {centros_dist}:"))

    for transportadora in lista_transportadoras:
        display(Markdown(f"              - {transportadora}"))

Percentual de entregas fora do prazo: 3,82 %

Percentual médio de atraso: 1,48 %

Centros de distribuições disponíveis: PR-Campina G. Sul, SP-Registro, SP-Jaguaré, SP-Cajamar

Transportadoras de cada centro de distribuição:

- PR-Campina G. Sul:

...

- SP-Cajamar:

              - Transportadora 1

              - Transportadora 2

              - Transportadora 3

              - Transportadora 5

# Análise Exploratória

## Entendimento entregas atrasadas sobre espaço temporal 

In [3]:
import hvplot.pandas
from bokeh.models.formatters import NumeralTickFormatter

df[["entregue_no_prazo", "dt_entrega_pedido"]].set_index("dt_entrega_pedido").groupby(
    [pd.Grouper(freq="ME"), "entregue_no_prazo"]
).size().hvplot.bar(
    title="Ocorrências Mensais Entregas no Prazo vs Fora do Prazo",
    xlabel="Mês",
    ylabel="Quantidade de Entregas",
    legend="top",
    width=1200,
    height=600,
).opts(
    xrotation=45, yformatter=NumeralTickFormatter(language="pt-br")
)

:Bars   [dt_entrega_pedido,entregue_no_prazo]   (0)

In [4]:
df.query("entregue_no_prazo == False")[["dt_entrega_pedido"]].set_index(
    "dt_entrega_pedido"
).groupby(pd.Grouper(freq="ME")).size().hvplot.bar(
    title="Ocorrências Mensais Entregas Fora do Prazo",
    xlabel="Data",
    ylabel="Quantidade de Eventos",
    width=1200,
    height=600,
)

:Bars   [dt_entrega_pedido]   (0)

## Entendimento quantitativo das entregas

In [5]:
from turtle import title
import hvplot.pandas
import holoviews as hv

df_total_grupo = df.groupby("des_cd_origem").size()
df_fora_prazo = df.query("entregue_no_prazo == False").groupby("des_cd_origem").size()

percentual_fora_prazo = pd.DataFrame(df_fora_prazo / df_total_grupo)
percentual_fora_prazo.sort_values(0, ascending=False, inplace=True)

percentual_fora_prazo.hvplot.bar(
    title="Percentual de Entregas Fora do Prazo de cada Centro de Distribuição",
    xlabel="Centro de Distribuição",
    ylabel="% Entregas Fora do Prazo",
    width=1200,
    height=600,
).opts(yformatter=NumeralTickFormatter(format="0%"))

:Bars   [des_cd_origem]   (0)

In [6]:
df.groupby("des_cd_origem").size().sort_values(ascending=False).hvplot.bar(
    title="Quantidade de Entregas por Centro de Distribuição",
    xlabel="Centro de Distribuição",
    ylabel="Quantidade de Entregas",
    width=1200,
    height=600,
).opts(yformatter=NumeralTickFormatter(language="pt-br"))

:Bars   [des_cd_origem]   (0)

In [7]:
df_total_grupo = df.groupby("grp_transportadora").size()
df_fora_prazo = (
    df.query("entregue_no_prazo == False").groupby("grp_transportadora").size()
)

percentual_fora_prazo = pd.DataFrame(df_fora_prazo / df_total_grupo)
percentual_fora_prazo.sort_values(0, ascending=False, inplace=True)

percentual_fora_prazo.hvplot.bar(
    title="Percentual de Entregas Fora do Prazo de cada Grupo de Transportadora",
    xlabel="Grupo de Transportadora",
    ylabel="% Entregas Fora do Prazo",
    width=1200,
    height=600,
).opts(yformatter=NumeralTickFormatter(format="0%"))

:Bars   [grp_transportadora]   (0)

In [8]:
import panel as pn

combo1 = pn.widgets.Select(
    name="Centro de Distribuição", options=sorted(df["des_cd_origem"].unique())
)


@pn.depends(combo1)
def grafico_barras_transportadoras_centro_origem(centro_dist):
    return (
        (
            df.query("des_cd_origem == @centro_dist")
            .groupby("grp_transportadora")
            .size()
            .sort_values(ascending=False)
        )
        .hvplot.bar(
            title=f"Quantidade de Entregas por Grupo de Transportadora - {centro_dist}",
            xlabel="Grupo de Transportadora",
            ylabel="Quantidade de Entregas",
            width=1200,
            height=600,
        )
        .opts(yformatter=NumeralTickFormatter(language="pt-br"))
    )


pn.Column(combo1, grafico_barras_transportadoras_centro_origem)

BokehModel(combine_events=True, render_bundle={'docs_json': {'8780e015-66a8-452a-b8ac-eb7f59d1d5ff': {'version…

In [9]:
import panel as pn

combo2 = pn.widgets.Select(
    name="Centro de Distribuição", options=sorted(df["des_cd_origem"].unique())
)


@pn.depends(combo2)
def grafico_barras_percent_atraso_transportadora(centro_dist):
    df_filtrado = df.query("des_cd_origem == @centro_dist")

    df_total_grupo = df_filtrado.groupby("grp_transportadora").size()
    df_fora_prazo = (
        df_filtrado.query("entregue_no_prazo == False")
        .groupby("grp_transportadora")
        .size()
    )

    percentual_fora_prazo = pd.DataFrame(df_fora_prazo / df_total_grupo)
    percentual_fora_prazo.sort_values(0, ascending=False, inplace=True)

    return percentual_fora_prazo.hvplot.bar(
        title=f"Percentual de Entregas Fora do Prazo por transportadora centro de distribuição {centro_dist}",
        xlabel="Centro de Distribuição",
        ylabel="% Entregas Fora do Prazo",
        width=1200,
        height=600,
    ).opts(yformatter=NumeralTickFormatter(format="0%"))


pn.Column(combo2, grafico_barras_percent_atraso_transportadora)

BokehModel(combine_events=True, render_bundle={'docs_json': {'6a536339-9d82-4f85-860c-e58306deed85': {'version…

In [10]:
df.query("pct_atraso > 1").groupby("des_cd_origem").agg(
    {"pct_atraso": "mean"}
).sort_values("pct_atraso", ascending=False).hvplot.bar(
    title="Percentual médio de atraso das entregas por Centro de Distribuição",
    xlabel="Centro de Distribuição",
    ylabel="Percentual de Atraso",
    width=1200,
    height=600,
).opts(
    yformatter=NumeralTickFormatter(format="0%", language="pt-br")
)

:Bars   [des_cd_origem]   (pct_atraso)

In [11]:
df.query("pct_atraso > 1").groupby("des_cd_origem").agg(
    {"qtde_dias_demora_despacho": "mean"}
).sort_values("qtde_dias_demora_despacho", ascending=False).hvplot.bar(
    title="Qtde média de dias para despacho das entregas fora do prazo por Centro de Distribuição",
    xlabel="Centro de Distribuição",
    ylabel="Qtde média de dias para despacho",
    width=1200,
    height=600,
).opts(
    yformatter=NumeralTickFormatter(format="0,0", language="pt-br")
)

:Bars   [des_cd_origem]   (qtde_dias_demora_despacho)

In [12]:
df.query("entregue_no_prazo == True").hvplot.box(
    y="pct_atraso",
    by="des_cd_origem",
    title="Distribuição Dias Despacho após pagamento por Centro de Distribuição entregas no prazo",
    xlabel="Centro de Distribuição",
    ylabel="Dias de Entrega",
    width=1200,
    height=600,
)

:BoxWhisker   [des_cd_origem]   (pct_atraso)

## Insights retirados da análise exploratória
Segue alguns insights retirados da análise exploratória:

- Minhas considerações:
  - Existe um percentual baixo de atrasos
  - Em média, os atrasos extrapolam em 48% do prazo da entrega
  - O centro de distribuição mais problemático é o de SP-Cajamar pelos seguintes motivos:
    - O que possui o maior atraso percentual em relação a todas as entregas efetuadas;
	- O que possui menor quantidade de entregas;
	- O que possui o maior tempo de despacho desde o pagamento para as entregas fora do prazo
	- O que possui maior dispersão na quantidade de dias de despacho após pagamento
- O centro de distribuição mais eficiente (SP-Registro) é o únco que não trabalha com o grupo de transportadora 5. Coincidência?
- Mesmo grupo de transportadora 5 no centro de distribuição SP-Cajamar, esse grupo é o que menos faz entregas nesse centro de distribuição 

- Solicitei ao ChatGPT uma análise e segue o que ele repondeu (**<font color="red">a ser validado, interpretado e entendido caso essas considerações sejam usadas</font>**)
  
| Tema | Evidência | Impacto |
|------|-----------|---------|
| **Taxa global de atraso** | 3,65 % dos 503 k pedidos; atrasados levam 1,48× o prazo | Atraso absoluto é baixo, mas severo quando ocorre |
| **Centro Distribuição crítico – SP-Cajamar** | 8,9 % de atraso | Averiguar motivo |
| **Despacho** | Atrasados: 2,6 d entre pagamento e envio; no-prazo: 1,7 d | Agilizar _picking_ pode cortar 30–40 % dos atrasos |
| **Transportadora 5** | 12,85 % de atraso (6× média) | Avaliar SLA ou contingência |
| **Praça “Capital”** | +0,9 d no despacho; 65 % mais atrasos | Investir em micro-hubs urbanos |
| **Ocorrências** | 14,6 % de atraso quando existe ocorrência | Automatizar tratamento preventivo |
